<span style="color: #E6E6E6;">
<span style="background-color: #00695C; padding:8px; display:block; border-left:4px solid #4682b4">

# Week 2 - Convolution (or filtering)

---


This week will focus on convolutions, mostly 2D, discrete convolutions. We will start by focusing on the "old-school" specific convolutions, before we in week 3 move onto learned convolutions used in CNN's.

The second, optional module, focuses on entropy and cross entropy, which may be useful for understanding some parts of next week, but as with all optional parts, will not be material for the exam.

**At the end of this week, you should be able to:**

- Explain how padding works to perserve image dimensions during convolution
- Explain different kinds of padding like constant, reflect, replicate, etc.
- Explain how 2d convolutions are performed on images
  - Be able to manually run or implement an algorithm that, given a kernel and an image or array, convolves the image with the kernel
- Have a rough understand of the differences between cross correlation and convolution.
- Know how you can construct different kind of kernels for specific purposes like blurring, edge enhancement, sharpening, etc.

**Optional**:
- Understand what entropy is and how it relates to the outcome probabilities of a random variable, for example when it is maximized or minimized
- Calculate the theoretical entropy of a given random variable
- Calculate the empirical entropy given random variates
- Understand how the cross-entropy relates to the entropy, for example when it is maximized or minimized
- Calculate the theoretical cross-entropy between two distributions
---


We are going to use images in the form of multidimensional arrays of data (a 2D matrix for a grayscale image and  3 2D matrices in case of a RGB image) and the kernel will be a 2D matrix (usually square with an odd number of rows/columns). Therefore, we will talk about 2D convolution.

If we have our input image $f_{in}$ (which we assume is of size $N \times N$ pixels) and a kernel K, the convolution is defined by:

$$f_{out}(x,y)=K*f_{in}(x,y)= \sum^N_{s=-N}\sum^N_{t={-N}}K(s,t)f_{in}(x-s,y-t)$$

where $f_{out}(x,y)$ is the value of the pixel $(x,y)$ of the output image $f_{out}$. \\

In the discrete case, which is what we consider, convolution considers the elementwise product between the **kernel** (sometimes called a filter), and the **sub-image** region considered. The following video shows how this can look with a given image and kernel. Note that the below case uses **mirror** or **symmetric** padding.

<div style="text-align: center;">
    <video controls src = "images/2dconv.mp4" width = "500" height = "400">
</div>

Also note, that in the below case, we assume the kernel **is flipped**. Though,since it is symmetric, we cannot tell the difference between it and a flipped kernel...

---

## Cross correlation or Convolution?

In these exercises we are technically going to consider **cross correlation** as a simplified technique of the convolution. Cross correlation is almost exactly the same as convolution, except here, we don't "flip" the kernel in our operation, leading to the formula looking like:

$$f_{out}(x,y)=K*f_{in}(x,y)= \sum^N_{s=-N}\sum^N_{t={-N}}K(s,t)f_{in}(x+s,y+t)$$

I.E: almost exactly the same, just with a $+$ where there used to be a "-". Practically, this equates to "flipping" the filter you pass over a signal, in our case an image. Practically, you don't need to worry about this whole "flipping of the kernel", it will rarely be relevant in machine learning, and mostly exists in pure signal processing. 

<!-- However, if you're curious, the below video shows the simple difference between cross correlation and convolution in a 1D case. (Note, that this video also shows the **continuous** case, as opposed to what we work with, which is the **discrete** case of convolution and cross correlation) -->

<!-- <div style="text-align: center;">
    <video controls src = "images/cross_corr_vs_conv_1d.mp4" width = "500" height = "400">
</div> -->

---

NOTE: ML engineers are lazy beyond what's right, and often don't care about the nuances of mathematics. In many ML libraries, cross correlation operations will be mislabeled as "convolution", despite not flipping the kernel. This is for example the case in [pytorch's "Conv2d"](https://docs.pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). The point is, don't worry too much about it when doing machine learning, just know the difference.

In these exercises, we will similarily be "lazy" and not ask you to flip the kernel, since this just makes it all that much easier.

</span>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

from scipy.signal import convolve2d as scipy_conv2d, convolve as scipy_conv1d, correlate2d
from time import time

<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 1: Understanding padding

It's a good idea to first get a handle on what padding actually is. Usually we work with the following types of padding:

- Reflect (sometimes called mirror)
- Edge (sometimes called replicate)
- Constant (a common special case i zero-padding)
- Wrap (sometimes called circular padding)

#### **1.1. Below the different padding types are shown in action using the np.pad function, give an explanation for what each type appears to do to the image**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **1.2. In the code, we don't explicitly state a value for the constant type pad, what does the default value appear to be? What does this equate to?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

In [ ]:
# Load the image
image_path = os.path.join("images", "kb_grayscale.jpg")
image_gray = np.array(Image.open(image_path))

# In this case, we use a fixed amount of padding for all dimensions of the image...
padding_size = [[100, 10], [30, 50]]  # ((top, bottom), (left, right))

pad_types = ["constant", "reflect", "wrap", "edge"]

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.ravel()

for ax, pad_type in zip(axes, pad_types):
    # Create padded image
    image_padded = np.pad(image_gray, padding_size, pad_type)

    # Display padded image    
    ax.imshow(image_padded, cmap="gray")
    ax.set_title(f"Padded: {pad_type}")

plt.tight_layout()
plt.show()


<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 2: Implementing convolution

A typical first exercise is implementing your own convolution function. Depending on how you do it, it can end up being a bunch of for loops and if-statements. While we won't work with 1d convolution untill around week 6, it is also a good exercise to implement it first to get a hang for how 2d convoltuion can be implemented. Remember, the formula for convolution is as follows:

$$f_{out}(x,y)=K*f_{in}(x,y)= \sum^N_{s=-N}\sum^N_{t={-N}}K(s,t)f_{in}(x-s,y-t)$$

And it needs to do what is essentially shown in the video:

<div style="text-align: center;">
    <video controls src = "images/2dconv.mp4" width = "500" height = "400">
</div>


#### **2.1. 💻 Complete the function below to implement 1d convolution, you should not add any padding to the final "image"**

#### **2.2. 💻 Complete the function below to implement 2d convolution, you should not add any padding to the final image**

#### **2.3. 💻 Test 1d and 2d convolution implementations against the package solution using the unittesting cell two cells below**

</span>

In [ ]:
def convolution1d(signal, kernel):
    """
    1d convolution function
    """

    assert kernel.shape[0] % 2 == 1, 'Kernel length must be odd for this convolution function'
    
    # Flip the kernel - So we don't need to worry about that...
    kernel = np.flip(kernel)
    
    # Get dimensions of both signal and kernel
    signal_length = signal.shape[0]
    kernel_length = kernel.shape[0]
    
    # Compute output length
    output_length = signal_length - kernel_length + 1
    
    # Initialize the array which will hold the output
    output = ...
    
    # Perform 1D convolution
    for i in range(output_length):

        # Get the sub-region of the signal that the kernel focuses on
        sub_region = ...
        
        # Multiply these with the corresponding values of the kernel
        sub_region_kernelized = ...

        # Sum these values and set the corresponding pixel in the output signal
        output[i] = ...
    
    return output
    

def convolution2d(image, kernel):
    assert kernel.shape[0] % 2 == 1 and kernel.shape[1] % 2 == 1, 'kernel must be an odd number for this convolution function'
    
    # Flip the kernel - So we don't need to worry about that...
    # Here we have to flip both "up-down" and "left-right"
    kernel = np.flipud(np.fliplr(kernel))
    
    # Get the dimensions of the image and the kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape
    
    # Compute the output dimensions
    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1
    
    # Initialize the output matrix which will hold the convolved pixel values
    output = np.zeros((output_height, output_width))
    
    # Perform 2D convolution
    # Loop over all values in the height of the output image...
    for i in range(output_height):
        # And all values in the width of the output image
        for j in range(output_width):
            # Get the 'sub-image' that the kernel currently focuses on
            sub_image = ...

            # Multiply each value in this sub image with the respective values of the kernel
            sub_image_kernelized = ...

            # Sum these values and set the corresponding pixel in the output image
            output[...] = ...
    
    return output

In [ ]:
import unittest

class Convolve2dtest(unittest.TestCase):

    def setUp(self):
        """"""
        self.own_conv_fun = convolution2d
        self.scipy_conv_fun = scipy_conv2d
        self.f_in = np.array([
            [0,1,0],
            [0,2,0],
            [0,3,0]
        ])
        self.w = np.array([[1,2,1]])

    def test_convolve2d_valid(self):
        """
        With "valid" padding in scipy_conv2d, it will create an image without padding it, possibly changing the output dimensions along the way
        
        TODO: Valid docstrings here

        """

        own_output = self.own_conv_fun(self.f_in, self.w)
        scipy_output = self.scipy_conv_fun(self.f_in, self.w, mode="valid")
        
        self.assertTrue(
                        np.all(own_output == scipy_output),
                        msg=f"Scipy output: \n {scipy_output} \n does not match own implementation output: \n {own_output}"
                        )

    def test_convolve2d_same(self):
        """
        If scipy_conv2d mode is set to "same" it will zero-pad to get the same number of output as input dimensions

        TODO: Valid docstrings here
        """

        # Pad the input image with enough columns to create a same output image
        f_padded = np.pad(self.f_in, [(0, 0), (1,1)], constant_values=0)

        own_output = self.own_conv_fun(f_padded, self.w)
        scipy_output = self.scipy_conv_fun(self.f_in, self.w, mode="same")
        
        self.assertTrue(
                        np.all(own_output == scipy_output),
                        msg=f"Scipy output with valid convolution: \n {scipy_output} \n does not match own implementation output: \n {own_output}"
                        )


class Convolve1dtest(unittest.TestCase):
    def setUp(self):
        """
        Set up the input signal and kernel for 1D convolution tests.
        Assign both the custom and reference convolution functions.
        """
        self.own_conv_fun = convolution1d
        self.scipy_conv_fun = scipy_conv1d

        # 1D input signal
        self.f_in = np.array([0, 1, 2, 3, 4])

        # 1D kernel
        self.w = np.array([1, 0, -1])

    def test_convolve1d_valid(self):
        """
        Test 1D convolution with 'valid' mode.
        This mode performs convolution without padding, resulting in a smaller output.
        """
        own_output = self.own_conv_fun(self.f_in, self.w)
        scipy_output = self.scipy_conv_fun(self.f_in, self.w, mode="valid")

        self.assertTrue(
            np.allclose(own_output, scipy_output),
            msg=f"Scipy output (valid):\n{scipy_output}\ndoes not match own output:\n{own_output}"
        )

    def test_convolve1d_same(self):
        """
        Test 1D convolution with 'same' mode.
        Since the custom function does not handle padding internally,
        we manually pad the input to match scipy's 'same' output dimensions.
        """

        # Calculate padding size: (kernel_size - 1) // 2 on each side
        pad_width = (self.w.size - 1) // 2
        f_padded = np.pad(self.f_in, (pad_width, pad_width), constant_values=0)

        own_output = self.own_conv_fun(f_padded, self.w)
        scipy_output = self.scipy_conv_fun(self.f_in, self.w, mode="same")

        self.assertTrue(
            np.allclose(own_output, scipy_output),
            msg=f"Scipy output (same):\n{scipy_output}\ndoes not match own output:\n{own_output}"
        )

class CustomTestResult(unittest.TextTestResult):
    def addSuccess(self, test):
        super().addSuccess(test)
        print(f"✅ SUCCESS: {test}")

    def addFailure(self, test, err):
        super().addFailure(test, err)
        print(f"❌ FAILURE: {test}\n{err[1]}")

    def addError(self, test, err):
        super().addError(test, err)
        print(f"⚠️ ERROR: {test}\n{err[1]}")

class CustomTestRunner(unittest.TextTestRunner):
    def _makeResult(self):
        return CustomTestResult(self.stream, self.descriptions, self.verbosity)


In [ ]:
try:
    print("Running tests to check own convolution1d function against scipy...")
    CustomTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Convolve1dtest))


    print("Running tests to check own convolution2d function against scipy...")
    CustomTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(Convolve2dtest))

except NameError:
    print("ERROR not find unit-testing suite, did you run the (hidden) cell right above?")



<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 3: Kernel design

When using convolution, we can manually design kernels that apply a certain effect to our images. This is still  large part of many image processing techniques still used today. Initially, we will work with three different kernels:

  $$ F_{blurry} = \frac{1}{9} \left[ {\begin{array}{ccc} 1&1&1\\1&1&1\\1&1&1 \end{array}} \right] \quad F_{sharp} = \left[ {\begin{array}{ccc}0&-1&0\\-1&7&-1\\0&-1&0\end{array}} \right] \quad F_{edge} = \left[ {\begin{array}{ccc}0&1&0\\1&-4&1\\0&1&0\end{array}} \right] $$

#### **3.1 The kernels obviously serve to either blur an image, sharpen an image, and enhance the edges of an image respectively. Give some inuition on why they have this effect. For example, why deos the first kernel blur the image?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">


Your answer here $\dots$

</span>


#### **3.2. 💻 Right now, all the kernels below do nothing to the image, rewrite them to correctly implement the blurry, sharpen, and edge kernels**

#### **3.3. 💻 What happens with the blurry kernel? Is there a way to make it blur more? Try implementing that**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **3.4. 💻 Using your own convolve2d function is probably a bit slow. Test the speed of your own implementation of convolution against that of scipy's. Potentially consider whether this can be improved**

#### **3.5. 💻 Play around a bit with the different images to try to answer some of the following questions**

- **What happens if you sharpen an image after blurring it?**
- **What happens if you edge-enhance an image after sharpening?**
- **How do the different kernels affect the maximum and minimum values of the image?**
- **How do kernels that affect the maximum and minimum values of an image appear to change the image visually? Is there a way to remidy this?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


</span>

In [ ]:
# Reload the image
image_path = os.path.join("images", "kb_grayscale.jpg")
image_gray = np.array(Image.open(image_path))

kernel_types = ["none", "blur", "sharp", "edge"]

fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.ravel()

# Define the different kernels
kernel_none = np.zeros((3,3))
kernel_none[1,1] = 1

kernel_blur = np.array([[0,0,0],[0,1,0],[0,0,0]])

kernel_sharp = np.array([[0, -1, 0], [-1, 7, -1], [0, -1, 0]])

kernel_edge = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])

kernels = [kernel_none, kernel_blur, kernel_sharp, kernel_edge]

# Normalize image to float values beteen 0 and 1
image_float = image_gray / float(np.max(image_gray))

time_taken = 0

for ax, kernel_type, kernel in zip(axes, kernel_types, kernels):
    t = time()
    # Perform convolution
    convolved_image = convolution2d(image_float, kernel)
    time_taken += time() - t

    # Display convolved image    
    ax.imshow(convolved_image, cmap="gray", vmin=0, vmax=1)
    ax.axis("off")
    ax.set_title(f"Convolved with {kernel_type}")

plt.tight_layout()
plt.show()

print(f"All convolutions completed after around {time_taken:.4f} seconds, can scipy beat this?")


<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 4: Sobel filter

In this exercise you are going to apply a more sophisticated edge detection filter than the one you used in the previous exercise. We will apply a Sobel filter to the original image. The Sobel filter (also known as Sobel operator https://en.wikipedia.org/wiki/Sobel_operator) is composed by these two different filters:

$$ F_{v} = \left[ {\begin{array}{ccc}-1&0&1\\-2&0&2\\-1&0&1\end{array}} \right] \quad F_{h} = \left[ {\begin{array}{ccc}-1&-2&-1\\0&0&0\\1&2&1\end{array}} \right]$$ 

These two filter, when convoluted with an image, are approximating the derivatives for the vertical and horizontal changes. Indeed, we have that we have that the value of the derivative is high when there is an high difference between neighbours pixels - this is exactly where an edge is. 

If we let $G_{v}$ and $G_{h}$ denote the resulting filtered images obtained by the convolution between $F_{v}$ and $F_{h}$ respectively with the original image, we can use them to compute an approximation of the gradient for each point in the image by computing a sort of "combined Sobel filter":

$$ G = \sqrt{ G_{v}^2 + G_{h}^2} $$


#### **4.1. 💻 Right now, all kernels are standard "nothing" kernels. Change these to be the sobel vertical, sobel horizontal, and combined sobel respectively**

#### **4.2. Comment on the resulting images.**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **4.3. How does the combined sobel filter for edge detection compare to the edge detection filter in the previous exercise? Why is it potentially inferior or superior?**


<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


#### **4.4. 💻 Remove the flipping of the kernel in your convolution function, thus changing it to cross-correlation, or use `scipy.signal.correlate2d`, which does not flip the kernel, how does the result change?** HINT: You may have to plot all three images below one another to see a difference, so just copy paste the whole cell with cross-correlation instead


<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

</span>

In [ ]:
# Reload the image
image_path = os.path.join("images", "kb_grayscale.jpg")
image_gray = np.array(Image.open(image_path))
image_float = image_gray / float(np.max(image_gray))

# Change Sobel kernels here to reflect true sobel kernel
sobel_kernel_v = np.array([[0,0,0],[0,1,0],[0,0,0]])
sobel_kernel_h = np.array([[0,0,0],[0,1,0],[0,0,0]])

# Create two convolutions by convolving image with the two sobel kernles
G_v = ...
G_h = ...

# Create combined Sobel image
G  = ...

sobel_names = ["Sobel Vertical", "Sobel Horizontal", "Combined Sobel"]
sobels = [G_v, G_h, G]

fig, axes = plt.subplots(1, 3, figsize=(20, 20))
axes = axes.ravel()

for ax, convolved_image, name in zip(axes, sobels, sobel_names):
    ax.imshow(convolved_image, cmap="gray", vmin=0, vmax=1)
    ax.axis("off")
    ax.set_title(name)

<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 5: RGB images

The exact same methods you've learned up until now can also be used on RGB images, or indeed images with **any number of channels**.

Note: Last time, we mentioned that an image with a single channel will appear gray. This is still true, even though there are single-channel plots below that appear red, green, and blue. The reason for this is just what color map we choose to plot with. In this case, we want to show the effect on each color individually, so we plot the single-channel images as if they are values that represent red, green, and blue instead of simply representing luminosity.


#### **5.1. 💻 Try using the blur, sharpen and edge enhancement kernel on each channel respectively. How does each channel react to convolution with these kernels?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


#### **5.2. 💻 Make a combined image by using each of the color channels after convolution, does this look the way you expect?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **5.3. 💻 Play around with mixing and matching different kernels across different channels and combining these in different ways.**


#### **5.4. Can you think of any cases where you would want to apply channel-wise convolutions independently, so convolution with one kernel to one channel, and another kernel to another channel?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


</span>

In [ ]:
# Load image
img_rgb_path = os.path.join("images", "field.png")
img_rgb = np.array(Image.open((img_rgb_path)))

img_rgb_float = img_rgb / img_rgb.max(axis=(0, 1)).astype(np.float32)

# Show the default image before convolution
plt.imshow(img_rgb_float, cmap="viridis")
plt.title(f"Original RGB image with shape {img_rgb.shape}")
plt.axis("off")
plt.tight_layout()
plt.show()

# Define the filters of the different channels
r_filter = kernel_sharp
g_filter = kernel_sharp
b_filter = kernel_sharp

# Extract the various channels
r_channel = img_rgb_float[:, :, 0]
g_channel = img_rgb_float[:, :, 1]
b_channel = img_rgb_float[:, :, 2]

# Convolve each channel with the given filter
r_fil = scipy_conv2d(r_channel, r_filter)
g_fil = scipy_conv2d(g_channel, g_filter)
b_fil = scipy_conv2d(b_channel, b_filter)


# Plot the result of each convolution
fig = plt.figure(figsize = (20, 30))
plt.subplot(4, 2, 1)
plt.imshow(r_channel, cmap="Reds", vmin=0, vmax=1)
plt.title("Red Channel")
plt.axis('off')

plt.subplot(4, 2, 2)
plt.imshow(r_fil, cmap="Reds", vmin=0, vmax=1)
plt.title("Red channel after convolution")
plt.axis('off')

plt.subplot(4, 2, 3)
plt.imshow(g_channel, cmap="Greens", vmin=0, vmax=1)
plt.title("Green channel")
plt.axis('off')

plt.subplot(4, 2, 4)
plt.imshow(g_fil, cmap="Greens", vmin=0, vmax=1)
plt.title("Green channel after convolution")
plt.axis('off')

plt.subplot(4, 2, 5)
plt.imshow(b_channel, cmap="Blues", vmin=0, vmax=1)
plt.title("Blue channel")
plt.axis('off')

plt.subplot(4, 2, 6)
plt.imshow(b_channel, cmap="Blues", vmin=0, vmax=1)
plt.title("Blue channel after convolution")
plt.axis('off')

plt.subplot(4, 2, 7)
channels = np.stack((r_channel, g_channel,b_channel,), axis=-1)
plt.imshow(channels, cmap="viridis", vmin=0, vmax=1)
plt.title("Original Image")
plt.axis('off')

# Sum the result of all convolutions
final = np.stack((r_fil, g_fil, b_fil), axis=-1)
plt.subplot(4, 2, 8)
plt.imshow(final, cmap="viridis", vmin=0, vmax=1)
plt.title("All channels after convolution")
plt.axis('off')

plt.tight_layout()
plt.show()

<span style="color: #E6E6E6;">
<span style="background-color: #00695C; padding:8px; display:block; border-left:4px solid #4682b4">

## $\star$ A primer on (Cross)-Entropy

*This is an optional subject*

Entropy can often best be considered as a measure of uncertainty, or an expected degree of "surprise" in observing the outcome of a random variable. The formula, as unintuitive as it is, is the following:

$$H(X) = - \sum^{N}_{i = 1}p(X = x_i) \log_2(p(X = x_i))$$

(Note here, that we consider only the discrete case of entropy.) (Also note, we use $log_2$, which we will write as $\log$ from hereon out)

The formula says, that the entropy of a random variable, is equal to the sum of the product of the probability for each possible outcome and the log of that probability.

This formula perhaps doesn't make much sense in and of itself, so instead it makes sense to consider the following example: We have our friend throw a fair six-sided die, which we then need to guess the result of using the least number of yes/no questions. Since the dice is fair, the best method here is constantly trying to **halve** the number of possible options the dice can land on. First by asking a question like *"Is the result 1, 2 or 3?"*. This strategy is illustrated below:

<p align="center">
 <img src="images/dice_throw_entropy.png" width="400"/>
</p>


As you can see, using this , we are sometimes lucky and get the correct answer after only 2 questions, however, somtimes we need to use 3. This brings us to the informal definition of entropy: 

<p align="center">
    <strong>Given that you ask the most informative questions possible, how many yes/no questions do you on average need to guess the result of random variable?</strong>
</p>


Do note, when we say "**most informative questions**", we mean that we will always try to ask the questions that most quickly bring us to a correct guess of the random variable, even if this means just guessing a value right off the bat. For example if our friend used a weighted die with a $50\%$ chance of landing on a 6, the most informative first question would be "*Is it 6?*" 


<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 6: Introductory entropy calculation

As always, it's useful to start wtih some manual calculations to get a feel for how the formulas actually work...

#### **6.1. Using the given formula, calculate the entropy of a fair, six-sided die**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


#### **6.2. Likewise, calculate the entropy of a fair, eight-sided die. How does this result compare to right above?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **6.3 Explain how Entropy can be seen as a degree of "surprise" associated with the outcome of random variables**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


#### **6.4. 💻 Complete the function in the cell below to calculate the entropy of a bernoulli random variable based on its p parameter. HINT: You can think of the bernoulli random variable as a variable denoting a sort of coin flip, with p being the probabilty of a getting heads.**


#### **6.5. 💻 Complete the function in the cell below to calculate the empirical entropy of a random variable based on recieved random variates**

#### **6.6. Examine the plot of the theoretical and empirical entropy, do they correspond to what you would expect? Why / Why not?** 

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


</span>

In [ ]:
def theoretical_bernoulli_entropy(p):
    """
    Compute the entropy of a bernoulli random variable based on its p parameter.

    Args:
        p (float): Parameter indicating the paobility of a success

    Returns:
        H (float): Entropy of the bernoulli random variable with the given parameter p
    """

    H = ...

    return H 

def empirical_entropy(random_variates):
    """
    Compute the empirical entropy of a set of random variates

    Args:
        random_variates (np.ndarray): Array of numbers indicating the outcome of a random variable

    Returns:
        H (np.ndarray): A numpy array containing the entropy of each of the random variables
    """
    # Compute the empirical probability mass function
    # So the probabilities of each outcome based the outcomes
    p = ...
    
    # Remove zero probabilities - in case any outcome is not represented
    p = p[p > 0]
    
    # Compute the entropy
    H = ...
    
    return H

In [ ]:
# Generate values for the p parameter in our bernoulli variable
p_values = np.linspace(0.001, 0.999, 500)

# Calculate the theoretical entropy of these values
entropy_values = theoretical_bernoulli_entropy(p_values)

# Generate random samples from a binomial distribution (numpy does not have bernoulli, as it is just a special case of the binomial anywaysl)
random_variates = np.array([np.random.binomial(1, p, 1000) for p in p_values])
# Calculate empirical entropies of the generated random variates by applying our function to each element in the array
empirical_entropies = list(map(empirical_entropy, random_variates))


# Plot both theoretical and empirical entropy
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# First subplot: Entropy as a function of p
axs[0].plot(p_values, entropy_values, label='Entropy H(p)')
axs[0].set_title('Theoretical entropy of a Bernoulli Random Variable')
axs[0].set_xlabel('Probability p')
axs[0].set_ylabel('Entropy H(p)')
axs[0].grid(True)

# Second subplot: Empirical entropy as a function of p
axs[1].plot(p_values, empirical_entropies, label='1 - p', color='r')
axs[1].set_title('Empirical entropy of a Bernoulli Random Variable')
axs[1].set_xlabel('Probability p')
axs[1].set_ylabel('Entropy H(p)')
axs[1].grid(True)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 7: Categorical entropy calculation

Calculating the entropy of a bernoulli random variable is maybe a bit trivial, but the same functions we just made, can be generalized to categorical variables.

#### **7.1. Run the cell and insepect the output what do you notice when comparing entropy across the different distributions?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


#### **7.2. Why can you say that the purple distribution has a lower degree of "surprise" on average?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **7.3 Imagine the categorical distributions represent the output probabilties of a ML classifier after being shown a datapoint with 4 possible classes, I.E: a probability of 25% for class 2, means that the classifier thinks there is a 25% probability that the datapoint belongs to class 2. Which of the categorical distributions would you as an ML engineer be most happy with?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


</span>

In [ ]:
# Define a few categorical distributions - Feel free to replace some of these with your own!
cat1 = np.array([0.1, 0.2, 0.3, 0.4])  # 4 categories
cat2 = np.array([0.25, 0.25, 0.25, 0.25])  # Uniform distribution
cat3 = np.array([0.05, 0.15, 0.6, 0.2])  # Slightly skewed towards category 3
cat4 = np.array([0.05, 0.05, 0.05, 0.85])  # Dominated by category 4

# Add to list and create dummy category names
categorical_probabilities = [cat1, cat2, cat3, cat4]
categories = np.arange(4)

# Draw random samples with the categorical distributions as probabilities
categorical_samples = [np.random.choice(categories, size=1000, p=cat) for cat in categorical_probabilities]

# Calculate empirical entropies
categorical_entropies = list(map(empirical_entropy, categorical_samples))


# Plot each categorical distribution along with its given empirical entropy
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for i, (axis, cat_probabilities, cat_entropy, color) in enumerate(zip(axes, categorical_probabilities, categorical_entropies, ['b', 'g', 'r', 'purple'])):
    axis.bar(categories, cat_probabilities, color=color)
    axis.set_title(f"Cat {i+1} Empirical Entropy: {cat_entropy:.2f}")
    axis.set_xlabel("Category")
    axis.set_ylabel("Probability")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()



<span style="color: #E6E6E6;">
<span style="background-color: #00695C; padding:8px; display:block; border-left:4px solid #4682b4">

## $\star$ Cross entropy

Now, we assume you at least have an idea of what entropy, lets move on to Cross-entropy, which looks much like the entropy:

$$H(p, q) = - \sum^N_{i = 1} p(x_i) \log(q(x_i))$$

What really changed? With the regular formula for entropy we compare a variable's probabilities to the log of themselves. Now, we compare the values of one probability distribution, $p$, to the values of a another probability distribution $q$.

We do not show it explicitly, but assume we have some random variable $X$, whose true probability distribution $p$ we attempt to approximate with some other probability distribution $q$.

Why is this useful? Two reasons mainly:

1. It gives us a very intuitive way of evaluating the performance of models that output probability distributions for classification, like neural networks.
2. It has some nice mathematical properties that directly ties it to multinomial logistic regression. (Which also gives it the cursed name of **"Negative Binomial Log-Likelihood"** (please don't ever call it that, it is the **Cross-Entropy**))

Because of its properties (especially, its relation to the "[Kullback-Liebler (KL) Divergence](https://johfischer.com/2021/12/31/intuitive-explanation-of-the-kullback-leibler-divergence/)"), the cross-entropy can be seen as measuring the "total cost" (in bits of information) of representing $p$ (the correct probability distribution) with $q$ (the approximate probability distribution)... optimal for modelling in ML. Why the **total** cost? Because the cross-entropy (unlike the KL-divergence) incorporates the intrinsic entropy of the variable in question. 

Indeed, the cross-entropy is actually "just" the sum of the KL divergence (how different our approximating distribution is from the true distribution), and the entropy (how uncertain the true distribution is). Therefore, when approximating a given distribution with some other distribution, the cross-entropy denotes how much uncertainty we are left with, or how much information we have to "pay" (usually paid in guesses) to get correct clasifications.

<span style="color: #E6E6E6;">
<span style="background-color: #545454; padding:8px; display:block; border-left:4px solid #4682b4">

### Exercise 8: Calculating cross entropy

In the cell below, we have a couple of categorical distributions, but this time, we have another *5th* distribution, used to approximate them. 

#### **8.1. In general, what appears to contribute to higher values of the cross entropy**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **8.2. Can we reasonably expect any of the original categorical distributions to always have a higher cross entropy, no matter the distribution used to approximate them?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>

#### **8.3. Subtract each distribution's entropy from the calculated cross-entropy, what do the results show? What does this demonstrate?**

<span style="background-color: #00590D; padding:8px; display:block; border-left:4px solid #4682b4">

Your answer here $\dots$

</span>


</span>

In [ ]:
def theoretical_cross_entropy(p, q):
    """
    Calculate the theoretical cross-entropy between two categorical distributions

    Args:
        p (list[float]): Probabilities for each catetory in the approximated distribution
        q (list[float]): Probabilities for each catetory in the approximating distribution 
    
    Returns:
        H (float): The theoretical cross entropy of the two input distributions
    """
    return -np.sum(p * np.log2(q))

In [ ]:
# Approximating distribtion - We will use this to calculate the cross entropy
cat_approx = np.array([0.1, 0.2, 0.3, 0.4])

# Define a few categorical distributions - Feel free to replace some of these with your own!
cat1 = np.array([0.1, 0.2, 0.3, 0.4])  # 4 categories
cat2 = np.array([0.25, 0.25, 0.25, 0.25])  # Uniform distribution
cat3 = np.array([0.05, 0.15, 0.6, 0.2])  # Slightly skewed towards category 3
cat4 = np.array([0.05, 0.05, 0.05, 0.85])  # Dominated by category 4

# Add to list and create dummy category names
categorical_probabilities = [cat1, cat2, cat3, cat4]
categories = np.arange(4)

# Calculate empirical entropies
categorical_cross_entropies = [theoretical_cross_entropy(categorical, cat_approx) for categorical in categorical_probabilities]

# Plot each categorical distribution along with its given empirical entropy
fig, axes = plt.subplots(3, 2, figsize=(12, 8))
axes = axes.ravel()

axes[0].bar(categories, cat_approx, color="mediumslateblue")
axes[0].set_title("Approximating distribution")
axes[0].set_xlabel("Category")
axes[0].set_ylabel("Probability")

for i, (axis, cat_probabilities, cat_entropy, color) in enumerate(zip(axes[1:], categorical_probabilities, categorical_cross_entropies, ['b', 'g', 'r', 'purple'])):
    axis.bar(categories, cat_probabilities, color=color)
    axis.set_title(f"Cat {i+1} Empirical Cross-Entropy: {cat_entropy:.2f}")
    axis.set_xlabel("Category")
    axis.set_ylabel("Probability")

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

<span style="color: #E6E6E6;">
<span style="background-color: #00695C; padding:8px; display:block; border-left:4px solid #4682b4">

## $\star$ Smallish note on *empirical* cross-entropy

When calculating cross-entropy for classifiers such as neural networks and multinomial logistic regression, we usually do not have the correct class probabilities $p$, but we do still have our approximate class probabilities $q$. As such, we simply change the class probabiltiies $p$ slightly, to simply be a one-hot vector, indicating what the correct class is. Our formula remains the same, but our probabilities look slighlty different:

$$
q_i = 
\begin{bmatrix}
    p_1 \\
    p_2  \\
    \vdots \\
    p_k \\
\end{bmatrix},

p_i =
\begin{bmatrix}
    0 \\
    1  \\
    \vdots \\
    0 \\
\end{bmatrix}
$$

Above shown in an example where the class of datapoint $i = 2$

This is a necessary distinction, as we have no simply way of making a "counterfactual" question asking "What is the probability that the datapoint $i$ would actually be representing as another class c?". This of course comes with its own host of quirks and downsides, one example being potentially pushing predictions towards extreme overconfidence, always predicting things with 100% accuracy, but in general, it works well enough that we still use it.

Bear in mind, many other explanations, like those on the [PyTorch documentation](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), for example, also include batch dimensions, which can change the formula just a little bit.